In [1]:
import datetime
import pandas as pd
import urllib2
import csv
import numpy as np
from matplotlib import pyplot as plt

In [2]:
def get_mta_df(source, original_format=True):
    if original_format:
        return get_mta_df_old(source)
    else:
        return get_mta_df_new(source)

def get_mta_df_old(source):
    # Import data format prior to 10/18/14
    col_names = ['C/A', 'UNIT', 'SCP', 'DATETIME', 'DESC', 'ENTRIES', 'EXITS']
    raw_data = urllib2.urlopen(source)
    reader = csv.reader((raw_line.replace('\0','') for raw_line in raw_data), delimiter=",")
    mta_list = list()
    for row in reader:
        for entry_num in range(0,8):
            offset = entry_num*5
            try:
                mta_list.append([row[0], row[1], row[2], 
                                 datetime.datetime.strptime('{} {}'.format(row[3+offset],
                                                                           row[4+offset]),
                                                            '%m-%d-%y %H:%M:%S'),
                                 row[5+offset], int(row[6+offset]), int(row[7+offset]),
                                ])
            except:
                pass
    mta_df = pd.DataFrame(mta_list, columns = col_names)
    return mta_df

def get_mta_df_new(source):
    # Import data format post 10/18/14
    col_names = ['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 
                 'DATE', 'TIME', 'DESC', 'ENTRIES', 'EXITS']
    mta_df = pd.read_csv(source, sep = ',', skiprows=1, header=None, names=col_names)
    mta_df['DATETIME'] = mta_df.apply(lambda x: 
                                      datetime.datetime.strptime('{} {}'.format(x.DATE,x.TIME),
                                                                 '%m/%d/%Y %H:%M:%S'), axis=1)
    return mta_df

def get_mta_df_by_date(date):
    original_format = date < datetime.datetime(2014,10,18)
    source_url_temp = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt'
    source_url = source_url_temp.format(date.strftime('%y%m%d'))
    return get_mta_df(source_url,original_format)

def get_mta_df_by_date_range(date, num_weeks):
    df_arry = []
    for week in range(0, num_weeks):
        run_date = date + datetime.timedelta(days = week*7)
        df_arry.append(get_mta_df_by_date(run_date))
    return pd.concat(df_arry)

def agg_by_station(target_df, stat_array):
    merged_df = merge_station(target_df)
    return merged_df.groupby('Station')[stat_array].sum()

def agg_by_station_date(target_df, stat_array):
    merged_df = merge_station(target_df)
    return merged_df.groupby(['Station','DATE'])[stat_array].sum()

def merge_station(target_df):
    return merge_station_strict(target_df)

def get_yankee_schedule():
    source = 'https://www.dropbox.com/s/2g5itrjc6mo4huu/yankee_home_2013.csv?dl=1'
    sched = pd.read_csv(source)
    # Restrict to night games
    sched = sched[sched['D/N']=='N']
    sched['Datetime'] = sched.apply(lambda x:
                                    datetime.datetime.strptime(x.Datetime,
                                                               '%m/%d/%Y %H:%M'),axis=1)
    sched['Date'] = sched.Datetime.dt.date
    return sched[['Date','Opp','Attendance']]

def get_station_table():
    source = 'http://web.mta.info/developers/resources/nyct/turnstile/Remote-Booth-Station.xls'
    station_table = pd.read_excel(source)
    return station_table

def merge_station_strict(target_df):
    station_table_raw = get_station_table()
    station_table = station_table_raw[['Remote','Booth','Station']].drop_duplicates()
    station_table = station_table.groupby(['Remote','Booth']).sum().reset_index()
    merged_df = target_df.merge(station_table,
                                left_on=['UNIT','C/A'],
                                right_on=['Remote','Booth'],
                                how='left')
    return merged_df

def merge_station_fuzzy(target_df):
    station_table_raw = get_station_table()
    station_table = station_table_raw[['Remote','Station']].drop_duplicates()
    station_table = station_table.groupby(['Remote']).sum().reset_index()
    merged_df = target_df.merge(station_table,left_on='UNIT',right_on='Remote',how='left')
    return merged_df

def calc_deltas(df):
    data_df = df[df.DESC=='REGULAR'].sort_values(['C/A','UNIT','SCP','DATETIME'])
    data_df_lag = data_df.groupby(['C/A','UNIT','SCP']).transform(lambda x:x.shift(-1))
    data_df.loc[:,'ENTRIES_end'] = data_df_lag['ENTRIES']
    data_df.loc[:,'EXITS_end'] = data_df_lag['EXITS']
    data_df['ENTRIES_delta'] = data_df.ENTRIES_end - data_df.ENTRIES
    data_df['EXITS_delta'] = data_df.EXITS_end - data_df.EXITS
    
    # Discard negative counts
    data_df.loc[data_df['ENTRIES_delta']<0,'ENTRIES_delta'] = 0
    data_df.loc[data_df['EXITS_delta']<0,'EXITS_delta'] = 0
    
    # Discard counts implying > 20 rotations per minute (20*60*4)
    data_df.loc[data_df['ENTRIES_delta']>4800,'ENTRIES_delta'] = 0
    data_df.loc[data_df['EXITS_delta']>4800,'EXITS_delta'] = 0
    
    return data_df

# Total Entries and Exits

What is the total number of entries & exits across the subway system for August 1, 2013?

In [4]:
data = get_mta_df_by_date(datetime.datetime(2013,8,3))
data_deltas = calc_deltas(data)
data_deltas = data_deltas[(data_deltas.DATETIME>=datetime.datetime(2013,8,1))&
                          (data_deltas.DATETIME<datetime.datetime(2013,8,2))]
data_deltas[['ENTRIES_delta','EXITS_delta']].sum()

ENTRIES_delta    5562793
EXITS_delta      4409520
dtype: float64

The subway system saw 5,562,793 entries and 4,409,520 exits across the subway system on August 1, 2013

Note turnstiles that reported less than zero / more than 20 rotations per minute over any four hour period were discarded.

# The busiest stations and turnstiles

Let’s define the busy-ness as sum of entry & exit count. What station was the busiest on August 1, 2013? What turnstile was the busiest on that date?

In [7]:
data = get_mta_df_by_date(datetime.datetime(2013,8,3))
data_deltas = calc_deltas(data)
data_deltas = data_deltas[(data_deltas.DATETIME>=datetime.datetime(2013,8,1))&
                          (data_deltas.DATETIME<datetime.datetime(2013,8,2))]
station_rollup = agg_by_station(data_deltas,['ENTRIES_delta','EXITS_delta'])
station_rollup['TOTAL_delta'] = station_rollup.ENTRIES_delta + station_rollup.EXITS_delta
station_rollup.sort_values('TOTAL_delta',ascending=False).head(3)

,ENTRIES_delta,EXITS_delta,TOTAL_delta
Station,,,
34 ST-PENN STA,175185,155858,331043
42 ST-GRD CNTRL,164823,151271,316094
34 ST-HERALD SQ,122548,114529,237077


The busiest station on August 1, 2013 was 34th Street – Penn Station

In [8]:
turnstile_rollup = data_deltas.groupby(['C/A','UNIT','SCP'])[['ENTRIES_delta','EXITS_delta']].sum()
turnstile_rollup['TOTAL_delta'] = turnstile_rollup.ENTRIES_delta + turnstile_rollup.EXITS_delta
turnstile_rollup.sort_values('TOTAL_delta',ascending=False).head(3)

,,,ENTRIES_delta,EXITS_delta,TOTAL_delta
C/A,UNIT,SCP,,,
N063A,R011,00-00-00,1882,9963,11845
R249,R179,01-00-09,964,10559,11523
R240,R047,00-00-00,3755,7302,11057


The busiest turnstile on August 1, 2013 was 00-00-00 at Times Square – 42nd Street / Port Authority Bus Terminal (N063A R011)

# The busiest stations in July

What were the busiest and least-busy stations in the system over all of July 2013?

In [10]:
data = get_mta_df_by_date_range(datetime.datetime(2013,7,6),5)
data_deltas = calc_deltas(data)
data_deltas = data_deltas[(data_deltas.DATETIME>=datetime.datetime(2013,7,1))&
                          (data_deltas.DATETIME<datetime.datetime(2013,8,1))]
station_rollup = agg_by_station(data_deltas,['ENTRIES_delta','EXITS_delta'])
station_rollup['TOTAL_delta'] = station_rollup.ENTRIES_delta + station_rollup.EXITS_delta
station_rollup.sort_values('TOTAL_delta',ascending=False).head(3)

,ENTRIES_delta,EXITS_delta,TOTAL_delta
Station,,,
34 ST-PENN STA,4577775,4018278,8596053
42 ST-GRD CNTRL,3848128,3610248,7458376
34 ST-HERALD SQ,3205652,2985543,6191195


The busiest station in August 2013 was 34th Street – Penn Station

In [11]:
station_rollup.sort_values('TOTAL_delta',ascending=False).tail(3)

,ENTRIES_delta,EXITS_delta,TOTAL_delta
Station,,,
ORCHARD BEACH,16260,1002,17262
BROAD CHANNEL,5772,2677,8449
AQUEDUCT TRACK,117,152,269


The least busiest station in August 2013 was Aqueduct Racetrack

# The busiest stations on Friday nights

Which station had the highest average number of entries between midnight & 4am on Fridays in July 2013?

In [18]:
data = get_mta_df_by_date_range(datetime.datetime(2013,7,6),5)
data_deltas = calc_deltas(data)
data_deltas = data_deltas[(data_deltas.DATETIME>=datetime.datetime(2013,7,1))&
                          (data_deltas.DATETIME<datetime.datetime(2013,8,1))&
                          (data_deltas.DATETIME.dt.time==datetime.time(0,0,0))&
                          (data_deltas.DATETIME.dt.dayofweek==4)]
station_rollup = agg_by_station(data_deltas,['ENTRIES_delta','EXITS_delta'])
station_rollup.sort_values('ENTRIES_delta',ascending=False).head(3)

,ENTRIES_delta,EXITS_delta
Station,,
42 ST-TIMES SQ,23885,6115
34 ST-HERALD SQ,11211,4441
34 ST-PENN STA,9361,5119


Times Square – 42nd Street / Port Authority Bus Terminal had the highest average number of entries between midnight & 4am on Fridays in July 2013.

# Usage Growth

What stations have seen the most usage growth/decline in the last year?

In [19]:
data_t0 = get_mta_df_by_date_range(datetime.datetime(2015,1,3),4)
data_t1 = get_mta_df_by_date_range(datetime.datetime(2016,1,2),4)

We can define station a few different ways. The most obvious is the self reported station name.

In [160]:
station_rollup_t0 = calc_deltas(data_t0).groupby('STATION')[['ENTRIES_delta','EXITS_delta']].sum()
station_rollup_t1 = calc_deltas(data_t1).groupby('STATION')[['ENTRIES_delta','EXITS_delta']].sum()
station_rollup = station_rollup_t0.join(station_rollup_t1, lsuffix='_t0', rsuffix='_t1', how='outer')
station_rollup = station_rollup.fillna(0)
station_rollup['YoY'] = station_rollup.ENTRIES_delta_t1 + station_rollup.EXITS_delta_t1 - station_rollup.ENTRIES_delta_t0 - station_rollup.EXITS_delta_t0
# Drop station names that do not exist in both periods
station_rollup[(station_rollup.ENTRIES_delta_t0+station_rollup.EXITS_delta_t0>0) &
               (station_rollup.ENTRIES_delta_t1+station_rollup.EXITS_delta_t1>0)].sort_values('YoY').head()

,ENTRIES_delta_t0,EXITS_delta_t0,ENTRIES_delta_t1,EXITS_delta_t1,YoY
STATION,,,,,
SUTPHIN BLVD,653496,507420,104683,62885,-993348
21 ST,203449,174611,39987,44831,-293242
VAN SICLEN AVE,215916,155170,74333,67641,-229112
34 ST-PENN STA,3977861,3389783,3884926,3269389,-213329
METS-WILLETS PT,144020,167220,49402,53041,-208797


By this metric, Sutphin Boulevard saw the largest drop. However this is largely because remote unit R024 was transferred from SUTPHIN BLVD to SUTPHIN-ARCHER in mid 2015. Similarly remote unit R303 was transferred from 21 ST to 21 ST-QNSBRIDGE and remote unit R434 was transferred from VAN SICLEN AVE to VAN SICLEN AV. The largest station drop with consistent remote units was 34th Street – Penn Station.

In [126]:
station_rollup[(station_rollup.ENTRIES_delta_t0+station_rollup.EXITS_delta_t0>0) &
               (station_rollup.ENTRIES_delta_t1+station_rollup.EXITS_delta_t1>0)].sort_values('YoY').tail(10)

,ENTRIES_delta_t0,EXITS_delta_t0,ENTRIES_delta_t1,EXITS_delta_t1,YoY
STATION,,,,,
WORLD TRADE CTR,287948,145531,335891,173800,76212
BOWLING GREEN,625253,578117,677530,615303,89463
PATH WTC 2,9950,26927,15662,204521,183306
DYCKMAN ST,167529,82718,328551,105554,183858
FULTON ST,1535037,1359578,1667897,1420392,193674
59 ST,1420518,1329697,1823457,1405696,478938
BROADWAY,86346,70922,403196,317784,563712
28 ST,805378,712336,1088218,1020814,591318
14 ST,935963,692746,1353799,1149621,874711


Similarly remote units R083, R453, and R203 were added to 23 ST from 23 ST-5 AVE and 23 ST-6 AVE. The largest station gain with consistent remote units and naming is PATH WTC 2.

For this reason we choose to match by control area + remote unit.

In [102]:
station_rollup_t0 = agg_by_station(calc_deltas(data_t0),['ENTRIES_delta','EXITS_delta'])
station_rollup_t1 = agg_by_station(calc_deltas(data_t1),['ENTRIES_delta','EXITS_delta'])
station_rollup = station_rollup_t0.join(station_rollup_t1, lsuffix='_t0', rsuffix='_t1', how='outer')
station_rollup['YoY'] = station_rollup.ENTRIES_delta_t1 + station_rollup.EXITS_delta_t1 - station_rollup.ENTRIES_delta_t0 - station_rollup.EXITS_delta_t0
station_rollup.sort_values('YoY').head()

,ENTRIES_delta_t0,EXITS_delta_t0,ENTRIES_delta_t1,EXITS_delta_t1,YoY
Station,,,,,
42 ST-PA BUS TE,2464347,1891440,2204050,1719541,-432196
57 ST-7 AVE,645657,396508,496571,208273,-337321
ROCKAWAY AVE,251385,196693,129160,74923,-243995
34 ST-PENN STA,3977861,3389783,3884926,3269389,-213329
METS-WILLETS PT,144020,167220,49402,53041,-208797


By this metric, we find that Times Square – 42nd Street / Port Authority Bus Terminal (which changed its name from 42 ST-PA BUS TE to 42 ST-PORT AUTH midyear) had the largest drop in ridership.

In [22]:
station_rollup.sort_values('YoY').tail()

,ENTRIES_delta_t0,EXITS_delta_t0,ENTRIES_delta_t1,EXITS_delta_t1,YoY
Station,,,,,
BOYD-88 ST,10407,1645,76025,25816,89789
SARATOGA AVE,132489,53927,201410,84654,99648
FULTON ST,1134795,970892,1267925,1018455,180693
PATH WTC 2,9950,26927,15662,204521,183306
MAIN ST,1238318,1067246,1393136,1151015,238587


While Main St (which changed its name from MAIN ST to FLUSHING-MAIN) had the largest gain.

# Capacity

What dates are the least busy? Could you identify days on which stations were not operating at full capacity or closed entirely?

In [189]:
data = get_mta_df_by_date_range(datetime.datetime(2015,1,3),10)
data_deltas = calc_deltas(data)
data_agg = data_deltas.groupby('DATE')[['ENTRIES_delta','EXITS_delta']].sum()
data_agg['TOTAL_delta'] = data_agg.ENTRIES_delta + data_agg.EXITS_delta
data_agg.sort_values('TOTAL_delta').head()

,ENTRIES_delta,EXITS_delta,TOTAL_delta
DATE,,,
01/27/2015,1378187,1137174,2515361
01/18/2015,2107901,1728133,3836034
02/15/2015,2296481,1871355,4167836
01/11/2015,2365236,1936583,4301819
03/01/2015,2385447,1924119,4309566


In general Sundays tend to be least busy. That said, over the first ten weeks of 2015, the lowest volume day was 01/27/2015 corresponding with a winter storm.

In [190]:
station_rollup = agg_by_station_date(data_deltas,['ENTRIES_delta','EXITS_delta'])
station_capacity = station_rollup.reset_index(level=1,drop=True).groupby(level=0).agg({'ENTRIES_delta':max,
                                                                                       'EXITS_delta':max})
station_utilization = station_rollup.div(station_capacity)

In [195]:
station_utilization[(station_utilization.ENTRIES_delta<0.01)|
                    (station_utilization.EXITS_delta<0.01)].head()

ENTRIES_delta  EXITS_delta
Station      DATE                                  
110 ST-CPN   01/31/2015       0.000000     0.003890
             02/07/2015       0.000000     0.003995
             02/08/2015       0.000000     0.001997
148 ST-LENOX 01/31/2015       0.000000     0.000240
             02/07/2015       0.000653     0.001198

To provide an example, 110 ST-CPN is operating at less than 1% capacity on 01/31/2015, 02/07/2015, and 02/08/2015 where capacity is measured as the maximum throughput observed on a single day. There are more than 250 instances over the first ten weeks of 2015 that meet this requirement.

# Predictive Modeling

Let’s develop a model for 4hr-interval exit count by turnstile device; What features were explored? How does this model perform? What is our predicted exit count for R195 during the 16:00-20:00 interval on Friday September 6, 2013?

While a general model would consider day of week, day of year, line, station, etc, R195 is the 161st Street – Yankee Stadium station, suggesting the MLB schedule may be a much more significant factor. Therefore we source the 2013 schedule from http://www.baseball-reference.com/teams/NYY/2013-schedule-scores.shtml.

In [197]:
yankee_schedule = get_yankee_schedule()

yankee_schedule.head()

,Date,Opp,Attendance
1,2013-04-03,BOS,40216
2,2013-04-04,BOS,40611
3,2013-04-12,BAL,35033
5,2013-04-14,BAL,34154
6,2013-04-16,ARI,34107


In [ ]:
data = get_mta_df_by_date_range(datetime.datetime(2013,8,3),6)
data_deltas = calc_deltas(data)
data_deltas = data_deltas[(data_deltas.UNIT=='R195')&
                          (data_deltas.DATETIME.dt.time==datetime.time(16,22,0))]
data_exits = data_deltas.groupby('DATETIME')[['EXITS_delta']].sum()
data_exits['DATE'] = data_exits.index.date
data_exits['DAY'] = data_exits.index.dayofweek
data_exits = data_exits.merge(yankee_schedule,left_on='DATE',right_on='Date',how='left')

Given Yankee attendance, we can predict station exits with a simple one factor model of station exits = 0.345 * attendance.

In [ ]:
data_model1 = data_exits.copy()
data_model1['ratio'] = data_model1.EXITS_delta / data_model1.Attendance
exits_to_attendance = np.average(data_model1[data_model1.Attendance>0].ratio)
data_model1['EXITS_pred'] = exits_to_attendance * data_model1.Attendance
data_model1['resid'] = data_model1.EXITS_delta - data_model1.EXITS_pred
data_model1[data_model1.Attendance>0][['DATE','EXITS_delta','DAY','EXITS_pred','resid']]

Our predicted exit count for R195 during the 16:00-20:00 interval on Friday September 6, 2013 is 15,237 versus 14,984 observed.

If attendance is not known, we can build a simple model based on the schedule and day of the week.

In [ ]:
data_exits.loc[data_exits.Opp.isnull(),'OffPeak'] = data_exits.loc[data_exits.Opp.isnull(),'EXITS_delta']
data_exits.loc[data_exits.Opp.isnull(),'Peak'] = data_exits.loc[~data_exits.Opp.isnull(),'EXITS_delta']

for act_level in ['Peak']:
    data_exits['t7'] = data_exits[act_level].shift(7)
    data_exits['t14'] = data_exits[act_level].shift(14)
    data_exits['t21'] = data_exits[act_level].shift(21)
    data_exits[act_level+'_pred'] = data_exits.t7
    data_exits.loc[data_exits[act_level+'_pred'].isnull(),act_level+'_pred']=data_exits.t14[data_exits[act_level+'_pred'].isnull()]
    data_exits.loc[data_exits[act_level+'_pred'].isnull(),act_level+'_pred']=data_exits.t21[data_exits[act_level+'_pred'].isnull()]

data_exits.loc[data_exits.Opp.isnull(),'EXITS_pred'] = data_exits.loc[data_exits.Opp.isnull(),'OffPeak_pred']
data_exits.loc[~data_exits.Opp.isnull(),'EXITS_pred'] = data_exits.loc[~data_exits.Opp.isnull(),'Peak_pred']

In [320]:
data_model2 = data_exits[['DATE','EXITS_delta','EXITS_pred']].copy()
data_model2['resid'] = data_model2.EXITS_delta - data_model2.EXITS_pred
data_model2

,DATE,EXITS_delta,EXITS_pred,resid
0,2013-07-27,2667,NaN,NaN
1,2013-07-28,1922,NaN,NaN
2,2013-07-29,3782,NaN,NaN
3,2013-07-30,3979,NaN,NaN
4,2013-07-31,3680,NaN,NaN
5,2013-08-01,3522,NaN,NaN
6,2013-08-02,3605,NaN,NaN
7,2013-08-03,2489,2667,-178
8,2013-08-04,2242,1922,320
9,2013-08-05,3761,3782,-21


In [322]:
data_exits

,EXITS_delta,DATE,DAY,Date,Opp,Attendance,OffPeak,Peak,t7,t14,t21,OffPeak_pred,Peak_pred,EXITS_pred
0,2667,2013-07-27,5,NaN,NaN,NaN,2667,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1922,2013-07-28,6,NaN,NaN,NaN,1922,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3782,2013-07-29,0,NaN,NaN,NaN,3782,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3979,2013-07-30,1,NaN,NaN,NaN,3979,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3680,2013-07-31,2,NaN,NaN,NaN,3680,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3522,2013-08-01,3,NaN,NaN,NaN,3522,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3605,2013-08-02,4,NaN,NaN,NaN,3605,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2489,2013-08-03,5,NaN,NaN,NaN,2489,NaN,NaN,NaN,NaN,2667,NaN,2667
8,2242,2013-08-04,6,NaN,NaN,NaN,2242,NaN,NaN,NaN,NaN,1922,NaN,1922
9,3761,2013-08-05,0,NaN,NaN,NaN,3761,NaN,NaN,NaN,NaN,3782,NaN,3782
